# INM706 - Deep Learning for Sequence Analysis

Authors: Laerte Adami - Elisa Troschka

In [1]:
with open ("hobbit.txt", "r", encoding='utf-8') as f:
     hobbit_text = f.read()

In [ ]:
hobbit_text

In [3]:
import nltk

In [1]:
from datasetHandler import FakeNewsDataset

In [2]:
data_dir = "Data"

NameError: name 'a' is not defined

In [15]:
import csv

with open('Data/True.csv') as csvfile:

    true = csv.DictReader(csvfile, fieldnames=['Title', 'Text', 'Subject', 'Date'])
    true.update(['Title', 'Text', 'Subject', 'Date'], Y)
    true["True"] = Y

    for row in reader:

        true["True"]

AttributeError: 'DictReader' object has no attribute 'update'